# 1) Cargamos los datos 📕

In [17]:
import pandas as pd
data = pd.read_csv("data/customer_dataset.csv", sep=',')

# 3) Discretización 📈➜📊 

In [18]:
#### Vamos a hacer una copia del dataframe data para luego evaluar si los puntos de corte se guardan y leen correctamente.
probando = data.copy()

# Para `orderAmount`
data['orderAmount'], saved_bins_order = pd.qcut(
    data['orderAmount'], 
    q=5, duplicates='drop', 
    retbins=True
    )
# Guardamos los puntos de corte
import pickle
with open('data/saved_bins_order.pickle', 'wb') as handle:
    pickle.dump(saved_bins_order, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
# Para `transactionAmount`
data['transactionAmount'], saved_bins_order = pd.qcut(
    data['transactionAmount'], 
    q=4, duplicates='drop', 
    retbins=True
    )
# guardo
with open('data/saved_bins_transaction.pickle', 'wb') as handle:
    pickle.dump(saved_bins_order, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# Vamos a evaluar que los puntos de corte / bins generado, se han guardado correctamente:
with open('data/saved_bins_order.pickle', 'rb') as handle:
    new_saved_bins_order = pickle.load(handle)
with open('data/saved_bins_transaction.pickle', 'rb') as handle:
    new_saved_bins_transaction = pickle.load(handle)

probando["orderAmount"] = pd.cut(
    probando['orderAmount'],
    bins=new_saved_bins_order, 
    include_lowest=True) # importante para que coincidan todos

probando['transactionAmount'], saved_bins_order = pd.qcut(
    probando['transactionAmount'], 
    q=4, duplicates='drop', 
    retbins=True
    )
probando.head(3)

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,"(9.999, 18.4]",pending,True,card,JCB 16 digit,Citizens First Banks,"(9.999, 21.0]",False,False,com,yahoo,only_letters,yes
1,"(18.4, 30.0]",fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,"(21.0, 34.0]",False,True,com,yahoo,only_letters,no
2,"(39.0, 47.0]",fulfilled,False,card,VISA 16 digit,Vertex Bancorp,"(34.0, 45.0]",False,False,com,yahoo,digits_and_letters,no


In [21]:
probando["orderAmount"].unique()

[(9.999, 18.4], (18.4, 30.0], (39.0, 47.0], (30.0, 39.0], (47.0, 353.0], NaN]
Categories (5, interval[float64, right]): [(9.999, 18.4] < (18.4, 30.0] < (30.0, 39.0] < (39.0, 47.0] < (47.0, 353.0]]

In [22]:
data["orderAmount"].unique()

[(9.999, 18.4], (18.4, 30.0], (39.0, 47.0], (30.0, 39.0], (47.0, 353.0], NaN]
Categories (5, interval[float64, right]): [(9.999, 18.4] < (18.4, 30.0] < (30.0, 39.0] < (39.0, 47.0] < (47.0, 353.0]]

In [23]:
data['transactionAmount'].unique()


[(9.999, 21.0], (21.0, 34.0], (34.0, 45.0], (45.0, 353.0]]
Categories (4, interval[float64, right]): [(9.999, 21.0] < (21.0, 34.0] < (34.0, 45.0] < (45.0, 353.0]]

In [24]:
probando["transactionAmount"].unique()


[(9.999, 21.0], (21.0, 34.0], (34.0, 45.0], (45.0, 353.0]]
Categories (4, interval[float64, right]): [(9.999, 21.0] < (21.0, 34.0] < (34.0, 45.0] < (45.0, 353.0]]

# 4) Preparación de datos 🔧

## 4.1) Intepretar los valores

### - paymentMethodIssuer

In [25]:
# Contar los elementos únicos que aparecen
data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
Bulwark Trust Corp.         29
Citizens First Banks        28
Grand Credit Corporation    27
Solace Banks                27
Rose Bancshares             25
B                            7
e                            5
c                            4
r                            3
                             2
n                            2
x                            2
o                            2
a                            1
p                            1
Name: count, dtype: int64

In [26]:
# Reemplazar un valor a la vez
weird_payment_method = ["B", "e", "c", "r", " ", "n", "x", "o", "a", "p"]
for payment_method in weird_payment_method:
    data['paymentMethodIssuer'] = data['paymentMethodIssuer'].replace(payment_method, 'weird')

data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
weird                       29
Bulwark Trust Corp.         29
Citizens First Banks        28
Solace Banks                27
Grand Credit Corporation    27
Rose Bancshares             25
Name: count, dtype: int64

### -paymentMethodProvider

data['paymentMethodProvider'].value_counts()


### -paymentMethodType

In [27]:
data['paymentMethodType'].value_counts()
from funpymodeling import status
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,orderAmount,145,0.232745,0,0.000000,5,category
1,orderState,145,0.232745,0,0.000000,3,object
2,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object
3,paymentMethodType,281,0.451043,0,0.000000,4,object
4,paymentMethodProvider,281,0.451043,0,0.000000,10,object
5,paymentMethodIssuer,281,0.451043,0,0.000000,11,object
6,transactionAmount,0,0.000000,0,0.000000,4,category
7,transactionFailed,0,0.000000,455,0.730337,2,bool
8,fraudulent,455,0.730337,107,0.171750,2,object
9,emailDomain,0,0.000000,0,0.000000,6,object


### - fraudulent
#### Vamos a hacer una especie de semáforo:
##### - False = Green le asignaremos el valor numérico 0
##### - True = Red le asignaremos el valor numérico 1
##### - Nan = Yellow le asignaremos el valor numérico 2

In [28]:
data['fraudulent']=data['fraudulent'].fillna(value="warning")
data["fraudulent"].value_counts()


fraudulent
warning    455
False      107
True        61
Name: count, dtype: int64

In [29]:
# Nos aseguramos que no exista ningún tipo de conflicto 
# por falta de compatibilidad entre str y bool de True/False.
data['fraudulent']=data['fraudulent'].astype(str)
class_map = {'False': 0, 'True': 1, 'warning': 2}
data['fraudulent'] = data['fraudulent'].map(class_map)
data.head(3)

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,"(9.999, 18.4]",pending,True,card,JCB 16 digit,Citizens First Banks,"(9.999, 21.0]",False,0,com,yahoo,only_letters,yes
1,"(18.4, 30.0]",fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,"(21.0, 34.0]",False,1,com,yahoo,only_letters,no
2,"(39.0, 47.0]",fulfilled,False,card,VISA 16 digit,Vertex Bancorp,"(34.0, 45.0]",False,0,com,yahoo,digits_and_letters,no


## 4.2) Custom tratamiento de datos faltantes
#### Lo hacemos para columnas categóricas, que en nuestro caso la única categórica con datos faltantes es `orderAmount`.

In [30]:
data['orderAmount'].unique()

[(9.999, 18.4], (18.4, 30.0], (39.0, 47.0], (30.0, 39.0], (47.0, 353.0], NaN]
Categories (5, interval[float64, right]): [(9.999, 18.4] < (18.4, 30.0] < (30.0, 39.0] < (39.0, 47.0] < (47.0, 353.0]]

In [31]:
data['orderAmount']=data['orderAmount'].cat.add_categories('desconocido')
data['orderAmount']=data['orderAmount'].fillna(value='desconocido')
data['orderAmount'].unique()

[(9.999, 18.4], (18.4, 30.0], (39.0, 47.0], (30.0, 39.0], (47.0, 353.0], 'desconocido']
Categories (6, object): [(9.999, 18.4] < (18.4, 30.0] < (30.0, 39.0] < (39.0, 47.0] < (47.0, 353.0] < 'desconocido']

# 5) One hot encoding ✂️
### Asigna el One hot encoding (ohe) a un nuevo dataframe llamado `data_ohe`

In [32]:
data_ohe = pd.get_dummies(data, dtype=int)
status(data_ohe)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,transactionFailed,0,0.0,455,0.730337,2,bool
1,fraudulent,0,0.0,107,0.171750,3,int64
2,"orderAmount_(9.999, 18.4]",0,0.0,527,0.845907,2,int32
3,"orderAmount_(18.4, 30.0]",0,0.0,527,0.845907,2,int32
4,"orderAmount_(30.0, 39.0]",0,0.0,525,0.842697,2,int32
5,"orderAmount_(39.0, 47.0]",0,0.0,526,0.844302,2,int32
6,"orderAmount_(47.0, 353.0]",0,0.0,532,0.853933,2,int32
7,orderAmount_desconocido,0,0.0,478,0.767255,2,int32
8,orderState_failed,0,0.0,573,0.919743,2,int32
9,orderState_fulfilled,0,0.0,223,0.357945,2,int32


In [33]:
data_ohe.head(3)

,transactionFailed,fraudulent,"orderAmount_(9.999, 18.4]","orderAmount_(18.4, 30.0]","orderAmount_(30.0, 39.0]","orderAmount_(39.0, 47.0]","orderAmount_(47.0, 353.0]",orderAmount_desconocido,orderState_failed,orderState_fulfilled,...,emailProvider_gmail,emailProvider_hotmail,emailProvider_other,emailProvider_weird,emailProvider_yahoo,customerIPAddressSimplified_digits_and_letters,customerIPAddressSimplified_only_letters,sameCity_no,sameCity_unknown,sameCity_yes
0,False,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
1,False,1,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,1,1,0,0
2,False,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,1,0,1,0,0


### Guardar One Hot Encoding


In [34]:
# Hay que eliminar nuestra columna objetivo de nuestro archivo pickle.
data_ohe_without_fraudulent = data_ohe.drop(["fraudulent"], axis=1)

In [35]:
# Guardar el nombre de las columnas
with open('data/categories_ohe_without_fraudulent.pickle', 'wb') as handle:
    pickle.dump(data_ohe_without_fraudulent.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)


# 6) Guardar dataset 💾
### Guardamos todo el One Hot Encoding, incluyendo nuestra variable objetivo: fraudulent

In [36]:
filename = "data/ohe_customer_dataset.csv"
data_ohe.to_csv(filename, index = False)